<a href="https://colab.research.google.com/github/van-dang/DMRI-FEM-Cloud/blob/master/MyTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About the code

In [0]:
# This program solves the Bloch-Torrey equation applied to computational diffusion MRI using 
# the finite element method coupled with the theta-method for the spatial discretization.

# The scope of usage: 
# (1) Single domains, Multilayered structures, manifolds
# (2) Membrane permeability for internal interfaces
#     Artificial permeability at the external interfaces
# (3) pure homogeneous Neumann BCs, (4) pseudo-periodic BCs

# Copyright (C) 2019 Van-Dang Nguyen (vdnguyen@kth.se)

# This file is part of DOLFIN.

# DOLFIN is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# DOLFIN is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU Lesser General Public License for more details.

# You should have received a copy of the GNU Lesser General Public License
# along with DOLFIN. If not, see <http://www.gnu.org/licenses/>.

# First added:  2017-10-10
# Last changed: 2019-04-25

# This demo is maintained by Van-Dang Nguyen
# Please report possible problems to vdnguyen@kth.se

# Setting a working environment with FEniCS

In [0]:
try:
    from google.colab import files
except:
    print("Running on a local runtime")
    
import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
    
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *

gmsh_dir=!which gmsh
if len(gmsh_dir)==0:
  !apt-get install gmsh
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

import sympy as sp


clear_output()

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

# Disable warnings
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("rothemain.rothe_utils")
logging.getLogger('UFL').setLevel(logging.WARNING)
logging.getLogger('FFC').setLevel(logging.WARNING)

import warnings
warnings.filterwarnings("ignore")


dolfin version: 2019.1.0


# Load the mesh

In [0]:
neuron_name='03b_spindle4aACC_dendrites_2';
neuron_dir='https://github.com/van-dang/RealNeuronMeshes/raw/master/volume_meshes/separated_spindles/03b_spindle4aACC_dendrites_2.msh.zip';
!wget -q $neuron_dir
!unzip -q $neuron_name''.msh.zip
!dolfin-convert $neuron_name''.msh $neuron_name''.xml
!sed -i 's/dim="2"/dim="3"/g' $neuron_name''.xml
mymesh = Mesh(neuron_name+'.xml');
                
gdim = mymesh.geometry().dim()
tdim = mymesh.topology().dim()

print('Topological dimension: ',tdim)
print('Geometrical dimension: ',gdim)
print('Mesh size: #vertices: '+str(mymesh.num_vertices()) +', #elements: '+str(mymesh.num_cells()) );

replace 03b_spindle4aACC_dendrites_2.msh? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
Converting from Gmsh format (.msh, .gmsh) to DOLFIN XML format
Expecting 5782 vertices
Found all vertices
Expecting 21209 cells
Found all cells
Conversion done
Topological dimension:  3
Geometrical dimension:  3
Mesh size: #vertices: 5782, #elements: 21209


# Mesh Modification

In [0]:
# Extract submesh inside a box
x0, y0, z0 = 0, 0, 0
d0, d1, d2 = 50, 50, 50
class InSideBox(SubDomain):
    def inside(self, x, on_boundary):
        cd1 = (abs(x[0]-x0) < d0);
        cd2 = (abs(x[1]-y0) < d1);
        cd3 = (abs(x[2]-z0) < d2);
        return cd1 and cd2 and cd3;
sub_domains = MeshFunction("size_t", mymesh, mymesh.topology().dim())
sub_domains.set_all(0)
insidebox = InSideBox();
insidebox.mark(sub_domains, 1)
submesh = SubMesh(mymesh, sub_domains, 1)

# Extract boundary mesh
boundarymesh = BoundaryMesh(submesh,"exterior")
V = FunctionSpace(boundarymesh, "CG", 1)
u = Function(V)
u.vector()[:]=1;
File("surface_mesh.pvd")<<u;



Links

In [0]:
# !ls /usr/share/dolfin/demo-python/documented/ 
# !cp /usr/share/dolfin/demo-python/documented/subdomains/* .
